# STAGE 1 scikit + ortools implementation


In [1]:
import trimesh as tm
import pyvista as pv
import numpy as np
import topogenesis as tg

import bisect # use interval?
import os
import copy
import math

from intervals import interval # use bisect?
from ortools.linear_solver import pywraplp
from skcriteria import Data, MIN, MAX

## loading lattice and context, setting default values

In [2]:
lattice_path = os.path.relpath("../data/voxelized_envelope.csv")
occ_lattice = tg.lattice_from_csv(lattice_path) # a default occupation lattice

path = os.path.relpath('../data/my_envelope.obj') # building outer boundaries file path
buildingplot = tm.load(path) # load specified building boundary mesh (footprint extruded to max height TODO: will be USER INPUT???)

env_all_vox = occ_lattice.flatten() # flattened lattice of all possible positions, True is occupied, False is unoccupied

# ^ TODO: make into dynamic lattice of current envelope

vox_size = env_all_vox.unit[0] 

max_extents_height = abs(buildingplot.bounds[0][2]) + abs(buildingplot.bounds[1][2])
max_extents_width = abs(buildingplot.bounds[0][0]) + abs(buildingplot.bounds[1][0])
max_extents_length = abs(buildingplot.bounds[0][1]) + abs(buildingplot.bounds[1][1])

In [3]:
base = buildingplot.apply_transform(tm.transformations.projection_matrix((0,0,0), (0,0,-1))) # project mesh on z plane to get footprint <-- easier way?
plot_area = base.area/2 # area calculates both sides of the flattened mesh, we need only 1
FSI_req = 4 # goal for the FSI TODO: to be decided by user

In [4]:
# Annual solar energy output of PV is calculated as follows:
# E(kwh) = A*r*H*PR where A = total solar panel area, r = solar panel yield/efficiency, H = Annual average solar radiation, PR = performance ratio
# from this site TODO find better source: https://photovoltaic-software.com/principle-ressources/how-calculate-solar-energy-power-pv-systems 

r = 0.15 # yield/efficiency estimate
H = 1050 # to be updated, currently wikipedia estimate. Slope, tilt, orientation may change values. kWh.y so maybe *365???
PR = 0.75 # default value, estimate
PV_req = 2 # kWh/m2/year TODO: change to actual expected value from included colors

## decision variables

In [5]:
# length int x>0
# width int x>0
# floor_count int x>1
# orientation float 0<x>360

# starting settings
f = 3 # number of floors
fh = 3.6 # floor height
l = 10 # envelope length
w = 20 # envelope width
h = f*fh # envelope height
deg = np.radians(225) # envelope rotation in radians

box = tm.creation.box((l,w,h))
box1 = copy.deepcopy(box) # copies are made to distinguish between original geometry (may be needed later) and new geometry
box2 = copy.deepcopy(box) # not sure if copying is necessary here TODO: find a way to create NEW instances directly by applying transforms instead of this

matrix = tm.transformations.rotation_matrix(deg, (0,0,1))
trans = tm.transformations.transform_around(matrix, box.centroid)

box3 = box1.apply_transform(trans)

# TODO: make variable variables - see below

## variables - OR-Tools

In [6]:
solver = pywraplp.Solver.CreateSolver("GLOP") # GLOP and SCIP get different results, also different results at different times --> different solvers? GUROBI?
infinity = math.inf

a = solver.IntVar(1.0, infinity, 'floors')
b = solver.IntVar(10.0, infinity, 'length')
c = solver.IntVar(10.0, infinity, 'width')
d = solver.IntVar(0.0, 90.0, 'rotation')


## constraints - OR-Tools

In [7]:
solver.Add(a * fh <= max_extents_height)
# solver.Add(b * math.cos(d.solution_value()) + c * math.sin(d.solution_value()) <= max_extents_length) # not sure if this works as it is supposed to right now
# solver.Add(c * math.cos(d.solution_value()) + b * math.sin(d.solution_value()) <= max_extents_width) # not sure if this works like it is supposed to right now

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x000001BED7E74720> >

## objective functions

In [8]:
def FSI(m2_plot, occupationlattice): # FSI based on the plot Area & all occupied cells in the lattice, for 2nd stage
    cell_count = sum(i for i in occupationlattice.flatten() if i != False) # <-- maybe rewrite? generator expression possible?
    m2_total = vox_size**2*cell_count
    FSI = m2_total/m2_plot
    return FSI

def FSI_lowres(floors, floor_percent=0.75): # quick FSI based on floorcount & plot coverage of the floors, for 1st stage
    FSI = floor_percent*floors
    return FSI

def FSI_normalize(FSI, target): # normalizes the achieved FSI to the target FSI as a score of 0 to 1
    FSI_score = 1/(1+(FSI-target)**2)
    return FSI_score

aa = FSI_normalize(FSI_lowres(10, (10*10)/plot_area), FSI_req)
print(aa)

0.060386007650769


## objective - OR-Tools

In [32]:
solver.Maximize(FSI_normalize(FSI_lowres(a.solution_value(), (b.solution_value() * c.solution_value())/plot_area), FSI_req)) # <-- not working yet, need to write functions inside maximize function
# solver.Maximize(1/(1+((a * ((b.solution_value() * c.solution_value())/plot_area))-FSI_req)**2)) # <-- also not working yet, need to write my own MILP solver? typeerror
# solver.Maximize(a) # <-- also not working yet, need to write my own MILP solver? typeerror <-- PROBLEM: can't multiply variables apparently?
# TODO: find way to linearize objective functions, OR use different method over MILP? --> get to NORMALIZED FSI/PV from current variables

# ALMOST got it: min/max goes wrong, does not want to add more floors

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('Solution:')
    print('Objective value =', solver.Objective().Value())
    print('floors =', a.solution_value())
    print('width =', b.solution_value())
    print('length =', c.solution_value())
    print('rotation =', d.solution_value())
else:
    print('The problem does not have an optimal solution.', status, pywraplp.Solver.OPTIMAL)



Solution:
Objective value = 0.05897708883622576
floors = 1.0
width = 10.0
length = 10.0
rotation = 90.0


AttributeError: 'Solver' object has no attribute 'GetIntegerParam'

In [10]:
def PV(occupationlattice, req = PV_req): # an accurate value (kWh estimate) of the PV potential of the mass <-- this currently uses uniform data for r, H, PR. 
    occ_shifted_down = np.roll(occupationlattice, (0,0,-1), (0,1,2)) # TODO: check calculation method, seems plausible
    roof_mask = (occ_lattice == True)*(occ_shifted_down == False)
    roof_mask_lattice = tg.to_lattice(roof_mask, occ_lattice)
    cell_count = sum(i for i in roof_mask_lattice.flatten() if i != False) # <-- maybe rewrite? generator expression possible?
    A = cell_count*vox_size**2
    E = A*r*H*PR
    PV_needed = A*PV_req
    PV_achieved = E
    if PV_achieved >= PV_needed: # TODO: Do I want this to be thresholds too? Or instead maximize
        PV_score = 1
    elif PV_achieved <= 0:
        PV_score = 0
    else:
        PV_score = PV_achieved/PV_needed
    return PV_achieved, PV_needed, PV_score

def PV_lowres(floors): # this gives the ratio between roof area and total area, for 1st stage TODO currently not valid/used
    PV = 1/floors
    return PV

def PV_normalize(PV, occupationlattice, target = PV_req): # this normalizes the achieved PV versus the expected amount of PV energy needed by the total floor area TODO: include in main PV def --> add here
    return PV

## visualisation

In [11]:
"""
# convert to trimesh definition, taken from spatial computing workshop
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the base mesh: light blue
p.add_mesh(tri_to_pv(box3), color='#abd8ff')
p.add_mesh(tri_to_pv(box), color = '#ab7ff')

# plotting
p.show(use_ipyvtk=True)
"""

"\n# convert to trimesh definition, taken from spatial computing workshop\ndef tri_to_pv(tri_mesh):\n    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)\n    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)\n    return pv_mesh\n\n# Visualize the mesh using pyvista plotter\n\n# initiating the plotter\np = pv.Plotter(notebook=True)\n\n# adding the base mesh: light blue\np.add_mesh(tri_to_pv(box3), color='#abd8ff')\np.add_mesh(tri_to_pv(box), color = '#ab7ff')\n\n# plotting\np.show(use_ipyvtk=True)\n"